In [1]:
# from seq2seq.models import AttentionSeq2seq
import numpy as np
SEQ_LENGTH = 40
SPEC_SYMB = '_'
pad = lambda s: s + (SEQ_LENGTH - len(s)) * SPEC_SYMB


def load_form_root(path):
    chars = []
    with open(path) as f:
        chars = sorted(list(set(f.read().lower().replace("\ufeff", "").replace("\t", "").replace("\n","") + SPEC_SYMB)))
    form_root = []
    with open(path) as morphs:        
        for line in morphs:        
            seq = list(map(pad, line.lower().replace("\ufeff", "").strip().split('\t')[1:]))
            # print(seq)
            for form in seq[1:]:
                root = seq[0]# align_from_root(form, seq[0])
                if root:
                    form_root.append((form, root))            
    return chars, form_root

def align_from_root(form, root):
    bound = root.find(SPEC_SYMB)
    idx = form.find(root[:bound])
    if idx > 0:
        return SPEC_SYMB*idx + root[:-idx]
    return None


# print(align_from_root("надмозг_____","мозг_____"))
# print("надмозг_____")

In [2]:
# Каждое слово дополняется до длины SEQ_LENGTH
SEQ_LENGTH = 40 
SPEC_SYMB = '_'
pad = lambda s: s + (SEQ_LENGTH - len(s)) * SPEC_SYMB




form_root = []
path = "../../Anglicizm/m_r.txt"


# with open(path) as morphs:
#     for line in morphs:        
#         form_root.append(tuple(map(pad, line.lower().replace("\ufeff", "").strip().split('\t')))) 

# chars = sorted(list(set(open(path).read().lower().replace("\ufeff", "").replace("\t", "").replace("\n","") + SPEC_SYMB)))

chars, form_root = load_form_root(path)

print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

print(chars)


# Каждый символ в слове пототм кодируется в one-hot вектор
print('Vectorization...')
X = np.zeros((len(form_root), SEQ_LENGTH, len(chars)), dtype=np.bool)
y = np.zeros((len(form_root), SEQ_LENGTH, len(chars)), dtype=np.bool)
for i, (f, r) in enumerate(form_root):
    for t, char in enumerate(f):
        X[i, t, char_indices[char]] = 1
    for t, char in enumerate(r):
        y[i, t, char_indices[char]] = 1
    
# [(f, r) for f, r in form_root if len(r) == 32]

total chars: 35
['-', '_', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё']
Vectorization...


In [3]:
# X = []
# y = []

# for i, (f, r) in enumerate(form_root):
#     X.append([char_indices(ch) for ch in f])
#     y.append([i, t, char_indices[char]] = 1
# len(form_root), path, chars

In [6]:
# !export LD_LIBRARY_PATH="$LD_LIBRARY_PATH:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64"
# !export CUDA_HOME=/usr/local/cuda
from __future__ import print_function
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Input, Bidirectional
from keras.layers.wrappers import TimeDistributed

Using TensorFlow backend.


In [7]:
# Параметры для сети
max_features = len(chars)
maxlen = SEQ_LENGTH  # cut texts after this number of words (among top max_features most common words)
batch_size = 64
hidden_units = 64

In [9]:
# Здесь непосредственно архитектура сети
model = Sequential()
model.add(Bidirectional(LSTM(hidden_units, return_sequences=True), batch_input_shape=(batch_size, maxlen, len(chars))))
model.add(Dropout(0.1))
model.add(TimeDistributed(Dense(len(chars), activation='softmax')))
model.compile('rmsprop', 'mse', metrics=['accuracy'])

/usr/local/lib/python3.5/dist-packages/keras/engine/topology.py:368: UserWarning: The `regularizers` property of layers/models is deprecated. Regularization losses are now managed via the `losses` layer/model property.
  warnings.warn('The `regularizers` property of '


In [ ]:
print('Train...')
model.fit(X, y,
          batch_size=batch_size, val
          nb_epoch=2, verbose=2)

Train...
Epoch 1/2


In [168]:
def get_root(form):
    s = pad(form)
    x = np.zeros((1,SEQ_LENGTH, len(chars)))
    for t, char in enumerate(s):
        x[0, t, char_indices[char]] = 1    
    y_x = model.predict(x)
    word = "".join([indices_char[i] for i in np.argmax(y_x[-1,:,:], axis=1)])
    word = word[:word.find(SPEC_SYMB)]
    return word


get_root('самсунговый')

'самсунг'